In [2]:
from pylab import *
import pandas as pd
from fastcluster import linkage

In [3]:
def sort_lidar_file_and_shape(file_name, width=2088, height=64):
    df = pd.read_csv(file_name,
                     names = ['x', 'y', 'z', 'intensity', 'ring', 'rotation', 'revolution'])
    df.sort_values(['rotation', 'ring'], inplace = True)
    img = np.zeros((width, height, 3))
    for i in range(width):
        df_rot_slice = df.iloc[height * i : height * (i + 1)]
        img[i] = df_rot_slice.as_matrix()[:, :3]
    
    lidar_readings = np.rot90(img)
    return lidar_readings

In [4]:
def calculate_normals(lidar_readings, width=2088, height=64):
    up_vectors = lidar_readings[1:] - lidar_readings[:lidar_readings.shape[0] - 1]
    left_vectors = np.zeros(up_vectors.shape)
    left_vectors[:, 1:] = lidar_readings[1:, 1:] - lidar_readings[1:, :lidar_readings.shape[1] - 1]
    left_vectors[:, 0] = lidar_readings[1:, 0] - lidar_readings[1:, lidar_readings.shape[1] - 1]
    normals = np.cross(up_vectors, left_vectors)

    averaged_normals = np.zeros(normals.shape)

    for i in range(0, height - 1):
        for j in range(0, width):
            center_normal = normals[i, j]
            avg_normal = np.zeros(3)
            count = 0
            for di in range(-1, 2):
                for dj in range(-1, 2):
                    new_i = i + di
                    new_j = j + dj
                    if new_i >= 0 and new_j >= 0 and new_i < height - 1 and new_j < width:
                        adjacent_normal = normals[new_i, new_j]
                        if (np.linalg.norm(center_normal - adjacent_normal) < 0.01):
                            avg_normal += adjacent_normal
                            count += 1
            averaged_normals[i, j] = avg_normal / count

    normals = averaged_normals.reshape((width * (height - 1), 3))  
    #normals = up_vectors.reshape((width * (height - 1), 3))
    return normals

In [5]:
readings_file_name = "1504892561136210918-cloudpoint.csv"

lidar_table = sort_lidar_file_and_shape(readings_file_name)
normals = calculate_normals(lidar_table)

lidar = lidar_table[1:].reshape((width * height, 3))

In [ ]:
width = 2088
height = 63 # Remove top row

order = np.zeros((height, width, 3))
total = width * height
green = np.array([0, 1, 0])
red = np.array([1, 0, 0])
for i in range(width):
    for j in range(height):
        val = (j) / height
        order[j, i] = val * red + (1 - val) * green

order = order.reshape((width * height, 3))

In [6]:
dists = np.linalg.norm(lidar, axis=-1)
inds = np.where(dists < 10)
 
lidar = lidar[inds]
normals = normals[inds]

In [ ]:
dendrogram = linkage (normals, method='average', metric='euclidean', preserve_input=True)